In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from pyspark.sql.functions import col, format_string, expr
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml import Pipeline
from pyspark.ml.feature import MinMaxScaler  # Add this import
from pyspark.ml.classification import LogisticRegression  # Add this import

In [13]:
# Initialize Spark session
spark = SparkSession.builder.appName("Milestone4").getOrCreate()

# Define the schema for the dataset
schema = StructType([
    StructField("winner_startingTrophies", IntegerType(), True),
    StructField("loser_startingTrophies", IntegerType(), True),
    StructField("winner_trophyChange", IntegerType(), True),
    StructField("loser_trophyChange", IntegerType(), True),
    StructField("winner_elixir_average", FloatType(), True),
    StructField("loser_elixir_average", FloatType(), True),
    StructField("battle_result", StringType(), True)  # Target column
])

# Define Paths for data and models
cleaned_data_path = "gs://my-project-bucket-ma/cleaned/"
trusted_data_path = "gs://my-project-bucket-ma/trusted/"
models_path = "gs://my-project-bucket-ma/models/"

# Load cleaned data
df = spark.read.csv(cleaned_data_path, header=True, schema=schema)

# Check label balance for 1's and 0's
# Use StringIndexer to convert battle_result to numeric labels (1 for winners, 0 for losers)
indexer = StringIndexer(inputCol="battle_result", outputCol="label", handleInvalid="skip")
df = indexer.fit(df).transform(df)

# Display the balance of labels (count of 1's and 0's)
label_counts = df.groupBy("label").count()
label_counts.show()

# Handle missing values by replacing nulls with default values
df = df.fillna({
    "winner_startingTrophies": 0,
    "loser_startingTrophies": 0,
    "winner_trophyChange": 0,
    "loser_trophyChange": 0,
    "winner_elixir_average": 3.8,
    "loser_elixir_average": 3.8
})

# Feature Engineering
# Assemble features into a single vector
assembler = VectorAssembler(
    inputCols=[
        "winner_startingTrophies", "loser_startingTrophies",
        "winner_trophyChange", "loser_trophyChange",
        "winner_elixir_average", "loser_elixir_average"
    ],
    outputCol="features"
)

# Scale features to a standardized range using MinMaxScaler
scaler = MinMaxScaler(inputCol="features", outputCol="scaled_features")

# Define Logistic Regression model
rf = RandomForestClassifier(featuresCol="scaled_features", labelCol="label", numTrees=100)

# Create a pipeline with assembler, scaler, and logistic regression model
pipeline = Pipeline(stages=[assembler, scaler, lr])

# Split data into training (80%) and testing (20%) sets
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

# Hyperparameter Tuning with Cross Validation
# Build a parameter grid to test different combinations of hyperparameters
paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [50, 100, 200]) \
    .addGrid(rf.maxDepth, [5, 10, 20]) \
    .addGrid(rf.minInstancesPerNode, [1, 2, 4]) \
    .build()

# Create a CrossValidator to tune hyperparameters using 5-fold cross-validation
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy"),
                          numFolds=5)

# Train the model using cross-validation
cvModel = crossval.fit(train_data)

# Evaluate the model on the test dataset
predictions = cvModel.transform(test_data)
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Cross-Validated Model Accuracy: {accuracy}")

# Save the best model from cross-validation
cvModel.bestModel.write().overwrite().save(models_path + "random_forest_cv_model")

# Save the processed dataset to the trusted folder
df.write.mode("overwrite").csv(trusted_data_path)

# Convert complex columns to strings for saving as CSV
predictions = predictions.withColumn("features_str", format_string("%s", col("features"))) \
                         .withColumn("scaled_features_str", format_string("%s", col("scaled_features")))

# Save predictions with necessary columns only
predictions_to_save = predictions.select("features_str", "scaled_features_str", "label", "prediction")
predictions_to_save.write.mode("overwrite").csv(trusted_data_path + "predictions")

# Save evaluation results to a local file
with open("/tmp/evaluation_results.txt", "w") as f:
    f.write(f"Cross-Validated Model Accuracy: {accuracy}")

# Copy the evaluation results to Google Cloud Storage
!gsutil cp /tmp/evaluation_results.txt {trusted_data_path}

24/12/12 03:44:21 WARN DAGScheduler: Broadcasting large task binary with size 1488.8 KiB
24/12/12 03:44:30 WARN DAGScheduler: Broadcasting large task binary with size 1488.2 KiB


+------+-----+
| label|count|
+------+-----+
| 934.0|    1|
|1765.0|    1|
| 576.0|    1|
| 389.0|    1|
|3667.0|    1|
| 594.0|    1|
|2561.0|    1|
|2024.0|    1|
|2316.0|    1|
|2425.0|    1|
|3441.0|    1|
|2324.0|    1|
|3463.0|    1|
|1514.0|    1|
| 180.0|    2|
| 387.0|    1|
|2139.0|    1|
|2113.0|    1|
|3391.0|    1|
| 542.0|    1|
+------+-----+
only showing top 20 rows



24/12/12 03:44:31 WARN DAGScheduler: Broadcasting large task binary with size 1490.0 KiB
24/12/12 03:44:57 WARN DAGScheduler: Broadcasting large task binary with size 1515.8 KiB
24/12/12 03:44:58 WARN DAGScheduler: Broadcasting large task binary with size 1519.1 KiB
24/12/12 03:44:58 WARN DAGScheduler: Broadcasting large task binary with size 1552.6 KiB
24/12/12 03:44:59 WARN DAGScheduler: Broadcasting large task binary with size 1553.7 KiB
24/12/12 03:44:59 WARN DAGScheduler: Broadcasting large task binary with size 1553.2 KiB
24/12/12 03:45:00 WARN DAGScheduler: Broadcasting large task binary with size 1554.4 KiB
24/12/12 03:45:00 WARN DAGScheduler: Broadcasting large task binary with size 1553.2 KiB
24/12/12 03:45:01 WARN DAGScheduler: Broadcasting large task binary with size 1554.4 KiB
24/12/12 03:45:01 WARN DAGScheduler: Broadcasting large task binary with size 1553.2 KiB
24/12/12 03:45:02 WARN DAGScheduler: Broadcasting large task binary with size 1554.4 KiB
24/12/12 03:45:02 WAR

24/12/12 03:45:20 WARN DAGScheduler: Broadcasting large task binary with size 1554.4 KiB
24/12/12 03:45:20 WARN DAGScheduler: Broadcasting large task binary with size 1553.2 KiB
24/12/12 03:45:20 WARN DAGScheduler: Broadcasting large task binary with size 1554.4 KiB
24/12/12 03:45:20 WARN DAGScheduler: Broadcasting large task binary with size 1553.2 KiB
24/12/12 03:45:21 WARN DAGScheduler: Broadcasting large task binary with size 1554.4 KiB
24/12/12 03:45:21 WARN DAGScheduler: Broadcasting large task binary with size 1553.2 KiB
24/12/12 03:45:21 WARN DAGScheduler: Broadcasting large task binary with size 1554.4 KiB
24/12/12 03:45:21 WARN DAGScheduler: Broadcasting large task binary with size 1553.2 KiB
24/12/12 03:45:22 WARN DAGScheduler: Broadcasting large task binary with size 1554.4 KiB
24/12/12 03:45:22 WARN DAGScheduler: Broadcasting large task binary with size 1553.2 KiB
24/12/12 03:45:22 WARN DAGScheduler: Broadcasting large task binary with size 1554.4 KiB
24/12/12 03:45:22 WAR

24/12/12 03:45:41 WARN DAGScheduler: Broadcasting large task binary with size 1554.4 KiB
24/12/12 03:45:41 WARN DAGScheduler: Broadcasting large task binary with size 1553.2 KiB
24/12/12 03:45:41 WARN DAGScheduler: Broadcasting large task binary with size 1554.4 KiB
24/12/12 03:45:41 WARN DAGScheduler: Broadcasting large task binary with size 1553.2 KiB
24/12/12 03:45:41 WARN DAGScheduler: Broadcasting large task binary with size 1554.4 KiB
24/12/12 03:45:42 WARN DAGScheduler: Broadcasting large task binary with size 1553.2 KiB
24/12/12 03:45:42 WARN DAGScheduler: Broadcasting large task binary with size 1554.4 KiB
24/12/12 03:45:42 WARN DAGScheduler: Broadcasting large task binary with size 1553.2 KiB
24/12/12 03:45:42 WARN DAGScheduler: Broadcasting large task binary with size 1554.4 KiB
24/12/12 03:45:42 WARN DAGScheduler: Broadcasting large task binary with size 1553.2 KiB
24/12/12 03:45:43 WARN DAGScheduler: Broadcasting large task binary with size 1554.4 KiB
24/12/12 03:45:43 WAR

...

24/12/12 05:11:35 WARN DAGScheduler: Broadcasting large task binary with size 1544.9 KiB
24/12/12 05:11:35 WARN DAGScheduler: Broadcasting large task binary with size 1543.7 KiB
24/12/12 05:11:35 WARN DAGScheduler: Broadcasting large task binary with size 1544.9 KiB
24/12/12 05:11:35 WARN DAGScheduler: Broadcasting large task binary with size 1543.7 KiB
24/12/12 05:11:36 WARN DAGScheduler: Broadcasting large task binary with size 1544.9 KiB
24/12/12 05:11:36 WARN DAGScheduler: Broadcasting large task binary with size 1543.7 KiB
24/12/12 05:11:36 WARN DAGScheduler: Broadcasting large task binary with size 1544.9 KiB
24/12/12 05:11:36 WARN DAGScheduler: Broadcasting large task binary with size 1543.7 KiB
24/12/12 05:11:36 WARN DAGScheduler: Broadcasting large task binary with size 1544.9 KiB
24/12/12 05:11:36 WARN DAGScheduler: Broadcasting large task binary with size 1543.7 KiB
24/12/12 05:11:37 WARN DAGScheduler: Broadcasting large task binary with size 1544.9 KiB
24/12/12 05:11:37 WAR

24/12/12 05:11:52 WARN DAGScheduler: Broadcasting large task binary with size 1544.9 KiB
24/12/12 05:11:52 WARN DAGScheduler: Broadcasting large task binary with size 1543.7 KiB
24/12/12 05:11:53 WARN DAGScheduler: Broadcasting large task binary with size 1544.9 KiB
24/12/12 05:11:53 WARN DAGScheduler: Broadcasting large task binary with size 1543.7 KiB
24/12/12 05:11:53 WARN DAGScheduler: Broadcasting large task binary with size 1544.9 KiB
24/12/12 05:11:53 WARN DAGScheduler: Broadcasting large task binary with size 1543.7 KiB
24/12/12 05:11:53 WARN DAGScheduler: Broadcasting large task binary with size 1544.9 KiB
24/12/12 05:11:54 WARN DAGScheduler: Broadcasting large task binary with size 1543.7 KiB
24/12/12 05:11:54 WARN DAGScheduler: Broadcasting large task binary with size 1544.9 KiB
24/12/12 05:11:54 WARN DAGScheduler: Broadcasting large task binary with size 1543.7 KiB
24/12/12 05:11:54 WARN DAGScheduler: Broadcasting large task binary with size 1544.9 KiB
24/12/12 05:11:54 WAR

Cross-Validated Model Accuracy: 0.006802721088435374


24/12/12 05:12:20 WARN DAGScheduler: Broadcasting large task binary with size 1704.9 KiB
24/12/12 05:12:38 WARN DAGScheduler: Broadcasting large task binary with size 1966.7 KiB


Copying file:///tmp/evaluation_results.txt [Content-Type=text/plain]...
/ [1 files][   52.0 B/   52.0 B]                                                
Operation completed over 1 objects/52.0 B.                                       
